In [165]:
import pandas as pd
import re

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)


In [166]:
df = pd.read_csv("posts.csv",delimiter=";",header=None, names=["date","post"], index_col=0)

In [167]:

df = df[["post"]]
df["post"] = df["post"].str.strip("gt.classof2025")
df["post"] = df["post"].str.replace("’","'")
df


,post
date,
2021-06-10T18:08:25.000Z,Hi everyone! Ik I'm late but I just committed ...
2021-06-10T18:06:44.000Z,Hi! My name is Sofia and I'm from Colombia. I'...
2021-06-10T18:05:03.000Z,Hey yall! im andrew and im from memphis tn. im...
2021-06-10T18:03:51.000Z,Hi everyone! I'm Ani and I'm from Garnet Valle...
2021-06-10T17:24:15.000Z,"Hey everyone! I'm Alex and i'm from Smyrna, wh..."
...,...
2020-12-21T22:55:08.000Z,"Hello everyone! My name is Claudia Vitale, and..."
2020-12-21T21:35:08.000Z,Hey guys:))) My name is Salma and I'm from Gwi...
2020-12-20T17:48:02.000Z,Hey everyone! My name is Jen O'Brien and I'm ...


In [168]:
def get_name(post):
    name = re.search(r"[mM]y [Nn]ame'?s ([A-z]* ?[A-z]*)",post)
    if name:
        return name.group(1).replace(" and", "").replace(" from", "")
    name = re.search(r"[mM]y [Nn]ame is ([A-z]* ?[A-z]*)",post)
    if name:
        return name.group(1).replace(" and", "").replace(" from", "")
    name = re.search(r"[Ii][' a]*m ([A-z]* ?[A-z]*)[ ,\.!]", post)
    if name:
        return name.group(1).replace(" and","").replace(" from", "")

def get_hometown(post, state=False):
    hometown = re.search(r"from ([A-z]*[ ,]*(([A-z]{2}[ ,.])|[A-z ]*))",post)
    if hometown:
        if state:
            return hometown.group(3)
        splitters = [" and",' but',' I ',' i ']
        hometown = hometown.group(1).strip(" ,")
        for splitter in splitters:
            hometown = hometown.split(splitter)[0]
        return hometown
    hometown = re.search(r"in ([A-z]*[ ,]*(([A-z]{2}[ ,.])|[A-z ]*))",post)
    if hometown:
        if state:
            return hometown.group(3)
        splitters = [" and",' but',' I ',' i ']
        hometown = hometown.group(1).strip(" ,")
        for splitter in splitters:
            hometown = hometown.split(splitter)[0]
        return hometown

def get_state(hometown):
    if not hometown:
        return None
    if re.search(r"[Aa]tlanta", hometown):
        return "GA"
    if re.search(r"[Gg]eorgia", hometown):
        return "GA"
    st = re.search(r"[, ]([A-z]{2})\.?$", hometown)
    if st:
        return st.group(1).upper()
    state = re.search(r" [A-Z]{2} ", hometown)

def get_major(post):
    major = re.search(r"major(?:ing)? in ([A-z]* [A-z]*)[ ,.]", post)
    if major:
        return major.group(1).strip(" and")
    major = re.search(r"([A-z]* [A-z]*) major", post)
    if major:
        return major.group(1).strip(" and")

def get_insta(post):
    insta = re.search(r"i?n?s?t?a?g?r?a?m?: @?([0-z.]*)",post, re.IGNORECASE)
    if insta:
        return insta.group(1).replace("sc:","").replace("Snap:","").replace("snap:","")

df["name"] = df["post"].apply(get_name)
df["hometown"] = df["post"].apply(get_hometown)
df["state"] = df["hometown"].apply(get_state)
df["major"] = df["post"].apply(get_major)
df['insta'] = df['post'].apply(get_insta)
df["gender"] = ["" for i in range(len(df))]

In [169]:

boys = open('boynames.txt', 'r')
boys = boys.read().split('\n')

girls = open('girlnames.txt', 'r')
girls = girls.read().split('\n')

def add_boy(name):
    f = open('boynames.txt','a')
    f.write(f"\n{name}")
    f.close()

def add_girl(name):
    f = open('girlnames.txt','a')
    f.write(f"\n{name}")
    f.close()

In [170]:
for i in range(len(df)):
    if not df["name"].iloc[i]:
        continue
    name = df["name"].iloc[i].split(' ')[0]
    name = name[0].upper() + name[1:].lower()
    if name in boys:
        df["gender"].iloc[i] = "B"
    elif name in girls:
        df["gender"].iloc[i] = "G"
    else:
        df["gender"].iloc[i] = "BLANK"

In [171]:



for i in range(len(df)):
    if not df["name"].iloc[i]:
        continue
    name = df["name"].iloc[i].split(' ')[0]
    name = name[0].upper() + name[1:].lower()
    if name in boys:
        df["gender"].iloc[i] = "B"
        continue
    elif name in girls:
        df["gender"].iloc[i] = "G"
        continue
    inp = input(f"{name}?(B/b/q/g/G) ")
    if not inp:
        continue
    inp = inp[0]
    if inp == "B":
        df["gender"].iloc[i] = "B"
        add_boy(name)
        boys.append(name)
    elif inp == "b":
        df["gender"].iloc[i] = "B"
    elif inp == 'g':
        df["gender"].iloc[i] = "G"
    elif inp == 'G':
        df["gender"].iloc[i] = "G"
        add_girl(name)
        girls.append(name)
    elif inp in ['q','Q']:
        break

In [172]:

df = df[df["gender"] != "G"]

df[["name","hometown","state","gender","major",'insta','post']].to_csv("analyzed.csv")